In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Banking dataset that helps running a campaign.
how do banks select target customers so that chances of selling their product maximize, this will help you understand these things.

Try to build a classification model that predicts if the customer will buy the product or not!
Age group:
10 - 19 = 1

20 - 29 = 2

30 - 39 = 3

40 - 49 = 4

50 - 59 = 5

60 - 69 = 6

70 - 79 = 7

80 - 89 = 8

90 - 99 = 9


age:
age of the customer.

age group:
what age group does the customer lie

eligible:
if the customer is eligible for the talk or not.

job:
what does the customer do?

salary:
salary of the customer

marital:
married or not?

education:
level of education complitrd

marital-education:
married or not- education.

targeted:
if the customer or being targeted or not

default:
if the customer in default list or not

balance:
remaining balance in their accounts

housing:
housing

loan:
has prior loan or not

contact:
source of this information

day:
today's date

month:
month

duration:
number of days

campaign:
marketing camoaign

pdays:
number of days that passed by after the client was last contacted

previous:
previous response

poutcome:
outcome of the previous marketing campaign

y:
yes or no

response:
response of the actor































In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve,f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics  import accuracy_score,classification_report,roc_auc_score,plot_roc_curve,plot_precision_recall_curve
from sklearn.decomposition import PCA
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_csv('/kaggle/input/bank-marketing/bank-marketing.csv')

In [ ]:
data

## EDA

In [ ]:
data.info()

In [ ]:
data.describe(include=np.number)

In [ ]:
data.describe(include=object)

In [ ]:
data.isnull().sum() # no missing value is found

In [ ]:
data.columns

In [ ]:
num_var=data.select_dtypes(include=np.number).columns.to_list()
cat_var=data.select_dtypes(include=object).columns.to_list()

In [ ]:
data1=data.copy()# just to retain the original data

### UNIVARIATE ANALYSIS

In [ ]:
i=1
plt.figure(figsize=(15,20))
for j in num_var:
    plt.subplot(5,3,i)
    sns.boxplot(data=data1,x=j)
    i+=1

#### we can see that the data is containing some outlier ,lets remove these outliers

In [ ]:
def outlier_tret(x):
    upper=x.quantile(0.98)
    lower=x.quantile(0.2)
    x=np.where(x>upper,upper,x)
    x=np.where(x<lower,lower,x)
    return x

In [ ]:
data1[num_var]=data1[num_var].apply(lambda x: outlier_tret(x))

In [ ]:
i=1
plt.figure(figsize=(15,20))
for j in num_var:
    plt.subplot(5,3,i)
    sns.boxplot(data=data1,x=j)
    i+=1

#### ouliers has been removed 

In [ ]:
data.hist(figsize=(15,15),color='red')
plt.show()

### BIVARIATE ANALYSIS

In [ ]:
sns.pairplot(data=data1)

In [ ]:
def barPerc(df,xVar,ax):
    '''
    barPerc(): Add percentage for hues to bar plots
    args:
        df: pandas dataframe
        xVar: (string) X variable 
        ax: Axes object (for Seaborn Countplot/Bar plot or
                         pandas bar plot)
    '''
    # 1. how many X categories
    ##   check for NaN and remove
    numX=len([x for x in df[xVar].unique() if x==x])

    # 2. The bars are created in hue order, organize them
    bars = ax.patches
    ## 2a. For each X variable
    for ind in range(numX):
        ## 2b. Get every hue bar
        ##     ex. 8 X categories, 4 hues =>
        ##    [0, 8, 16, 24] are hue bars for 1st X category
        hueBars=bars[ind:][::numX]
        ## 2c. Get the total height (for percentages)
        total = sum([x.get_height() for x in hueBars])

        # 3. Print the percentage on the bars
        for bar in hueBars:
            ax.text(bar.get_x() + bar.get_width()/2.,
                    bar.get_height(),
                    f'{bar.get_height()/total:.0%}',
                    ha="center",va="bottom")

In [ ]:
i=1
plt.figure(figsize=(20,80))
for col in cat_var:
    plt.subplot(9,2,i)
    ax=sns.countplot(data=data1,x=col,hue='y')
    barPerc(df=data1,xVar=col,ax=ax)
    plt.xticks(rotation=45)
    i+=1

#### from above plot we can make an inference about the  specific type of customer who are more willing to buy the product.
#### So we will choose only those category where the yes% is high, means more % of customer in that category are willing to buy the product 
 
#### 1- eligible         : No 
#### 2- job              : student
#### 3- marital          : single
#### 4- education        : tertiary
#### 5-marital education :married tertiary,single secondary
#### 6-targeted          : NO
#### 7-default           : NO
#### 8-housing           :NO
#### 9-loan               :NO
#### 10-contact           :cellular
#### 11-month             : april
#### 12-poutcome          : success
#### So these are the group of customer where  they is more chance of getting sell of product ,so we should first target them.

In [ ]:
num_var

In [ ]:
i=1
plt.figure(figsize=(15,20))
for j in num_var:
    plt.subplot(5,2,i)
    sns.boxplot(data=data1,x='y',y=j)
    i+=1
    

#### the mean age of the group of customer who is buyer of product yes or no ,is seem to be the equal ,so lets apply t-test to check whether mean age  for both category is same  or not

In [ ]:
age_yes=data1.groupby(by='y').get_group('yes')[['age']]
age_no=data1.groupby(by='y').get_group('no')[['age']]
p=stats.ttest_ind(age_no,age_yes)
print(age_yes.mean())
print(age_no.mean())
print(p)

#### we get p-value<.05,which means that the mean age of both group is significantly different ,so the higher age person are more likely to buy the product. 

In [ ]:
age_group_yes=data1.groupby(by='y').get_group('yes')[['age group']]
age_group_no=data1.groupby(by='y').get_group('no')[['age group']]
p=stats.ttest_ind(age_group_no,age_group_yes)
print(age_group_yes.mean())
print(age_group_no.mean())
print(p)

In [ ]:
ax=sns.countplot(data=data1,x='age group',hue='y')
barPerc(df=data1,xVar='age group',ax=ax)

#### p-value is less than 0.05, which means that both group of age group is different, the age group 6 -(60-69) is more likely to buy the product

In [ ]:
salary_yes=data1.groupby(by='y').get_group('yes')[['salary']]
salary_no=data1.groupby(by='y').get_group('no')[['salary']]
p=stats.ttest_ind(salary_no,salary_yes)
print(salary_yes.mean())
print(salary_no.mean())
print(p)

#### here p-value is less than 0.05, so the mean salary for the both group (yes ,no) is significantly different,so we coclude that higher salary earning customer are more willing to buy the product

In [ ]:
balance_yes=data1.groupby(by='y').get_group('yes')[['balance']]
balance_no=data1.groupby(by='y').get_group('no')[['balance']]
p=stats.ttest_ind(balance_no,balance_yes)
print(balance_yes.mean())
print(balance_no.mean())
print(p)

#### customer having higher balance is more willing to by the product

In [ ]:
duration_yes=data1.groupby(by='y').get_group('yes')[['duration']]
duration_no=data1.groupby(by='y').get_group('no')[['duration']]
p=stats.ttest_ind(duration_no,duration_yes)
print(duration_yes.mean())
print(duration_no.mean())
print(p)

#### higher the duration ,higher the chance of product selling

In [ ]:
cam_yes=data1.groupby(by='y').get_group('yes')[['campaign']]
cam_no=data1.groupby(by='y').get_group('no')[['campaign']]
p=stats.ttest_ind(cam_no,cam_yes)
print(cam_yes.mean())
print(cam_no.mean())
print(p)

#### lower the marketing campaign,higher the chance of selling product
#### greater the previous campaign response greater the chance of selling the product

In [ ]:
data2=data1.copy() # to retain the data1

## FEATURE ENGINEERING

#### age and age group are the same so we will keep only age

In [ ]:
data2.drop(columns='age group',inplace=True)

In [ ]:
data2=pd.get_dummies(data=data2,columns=cat_var,drop_first=True)

In [ ]:
data2

In [ ]:
corr=data2.corr()[['y_yes']].sort_values(by='y_yes',ascending=False)

In [ ]:

corr.plot(kind='bar',figsize=(15,8))
plt.show()

#### here we can see the correlation of different variables with y_yes

In [ ]:
sc=StandardScaler()

In [ ]:
num_var.remove('age group') # removing age group

In [ ]:
scaled=sc.fit_transform(data2[num_var])
scaled=pd.DataFrame(scaled,columns=num_var)
scaled_data=pd.concat([scaled,data2.drop(columns=num_var)],axis=1)

In [ ]:
scaled_data1=scaled_data.drop(columns='y_yes')

In [ ]:
scaled_data1

In [ ]:
pca=PCA()

In [ ]:
model=pca.fit_transform(scaled_data1)
model

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

In [ ]:
feature=range(pca.n_components_)

In [ ]:
plt.figure(figsize=(20,5))
plt.bar(feature,pca.explained_variance_ratio_)
plt.step(feature,np.cumsum(pca.explained_variance_ratio_),color='red')
plt.axhline(y=.95,color='green')
plt.grid(axis='x')
plt.text(0.5,.85,'95% cutt-off thresold',color='green',fontsize=15)
plt.show()

In [ ]:
pca1=PCA(n_components=25,whiten=True)

In [ ]:
model=pca1.fit_transform(scaled_data1)

In [ ]:
np.cumsum(pca1.explained_variance_ratio_)

### MODEL BUILDING

In [ ]:
Y=scaled_data[['y_yes']]

In [ ]:
Y.value_counts()# ratio is 1:8 so it is not imbalance classification problem

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(model,Y,test_size=.3,random_state=0)

In [ ]:
lg=LogisticRegression()
dt=DecisionTreeClassifier()
rf=RandomForestClassifier()
nb=GaussianNB()
knn=KNeighborsClassifier()
gbc=GradientBoostingClassifier()
adb=AdaBoostClassifier()
sgd=SGDClassifier()
svc=SVC()
mlpc=MLPClassifier()

In [ ]:
algo=[lg,dt,rf,nb,knn,adb,sgd,svc,mlpc]

In [ ]:
for i in algo:
    i.fit(x_train,y_train)
    acc=i.score(x_test,y_test)
    print(acc,i)

#### we choose decision tree classifier model

In [ ]:
dt1=DecisionTreeClassifier()

In [ ]:
dt1.fit(x_train,y_train)

In [ ]:
y_pred=dt1.predict(x_test)

In [ ]:
print(classification_report(y_pred,y_test))

In [ ]:
plot_roc_curve(dt1,x_test,y_test)

In [ ]:
plot_precision_recall_curve(dt1,x_test,y_test)